## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from batman import batman_curve

%load_ext autoreload
%autoreload 2

## Create random points

In [ ]:
sample_size = 10000
X1 = 16*np.random.random(size=(sample_size, 1)) - 8
X2 =  8*np.random.random(size=(sample_size, 1)) - 4
X = np.concatenate((X1, X2), axis=1)
Y = np.zeros((sample_size, 1))

for i in range(len(X)):
    curr_cat = batman_curve(X[i, 0], X[i, 1])
    Y[i, 0] = curr_cat

color = list(map(lambda x: 'y' if x == 1 else 'k', Y))

## Show scatter plot

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], color=color)

## Cleaner figure

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], color=color)
plt.axis('off')
plt.savefig('resources/batman-logo.png', dpi=300, bbox_inches='tight')